In [26]:
import json, requests
import Levenshtein as lev
import unicodedata
import pandas as pd
import urllib
import os, os.path
import time

In [7]:

ID = "aaf20ac1"
key = "7bd88c15c64569da8de3c414ace8d236"

baseURL = "http://api.yummly.com/v1/api/recipes?_app_id="+ID+"&_app_key="+key+"&q=@&maxResult=51"
baseURLGet = "http://api.yummly.com/v1/api/recipe/@?_app_id="+ID+"&_app_key="+key

noImages = []
csvList = []
jsonBackup = {}

In [8]:
def convertName(name):
    converted = unicodedata.normalize('NFKD', name).encode('ascii','ignore')
    return converted.lower()

def bestMatch(query, response):
    tmpQuery = query.lower()
    levDistances = [lev.distance(convertName(match["recipeName"]), tmpQuery) for match in response["matches"]]
    return response["matches"][levDistances.index(min(levDistances))]

def getRecipe(candidateID):
    url = baseURLGet.replace("@", candidateID)
    response = json.loads(requests.get(url).text)
    return response

def buildJson(original, searchResponse, recipeResponse):
    jsonBuilder = {}

    jsonBuilder["RecipeURL"] = recipeResponse["source"]["sourceRecipeUrl"]
    jsonBuilder["CleanIngredients"] = searchResponse["ingredients"]
    jsonBuilder["RawIngredients"] = recipeResponse["ingredientLines"]
    jsonBuilder["RecipeName"] = searchResponse["recipeName"]
    jsonBuilder["FoodName"] = original
    return jsonBuilder

def rangeMatch(query, response):
    tmpQuery = query.lower()
    levDistances = [lev.distance(convertName(match["recipeName"]), tmpQuery) for match in response["matches"]]
    response["matches"].pop(levDistances.index(min(levDistances)))
    return response["matches"]

def saveImageFromUrl(query, recipeName, index, lstURL):
    tmpQuery = (query.replace(" ", "_")).lower()
    tmpRecipeName = (recipeName.replace(" ", "_")).lower()
    tmpRecipeName = unicodedata.normalize('NFKD', tmpRecipeName).encode('ascii','ignore')
    
    candidate = "hostedLargeUrl"
    try:
        directory = "food101Pictures/"+tmpQuery
        if not os.path.exists(directory):
            os.makedirs(directory)

        fileName = directory+"/"+str(index)+"_"+tmpRecipeName+"_"+candidate+".jpg"
        urllib.urlretrieve(lstURL[0][candidate], fileName)
        csvList.append((fileName, recipeName))
        return fileName
    except:
        print "Something went wrong and could not retrieve the image for "+query+" recipeName: "+recipeName+" and index "+str(index)
        return None

def checkIfImage(response):
    try:
        response[0]["hostedLargeUrl"]
        return True
    except:
        return False
    
def buildWideJson(original, searchResponse, recipeResponse, index):
    jsonBuilder = {}
    if recipeResponse["images"] and checkIfImage(recipeResponse["images"]):
        fileName = saveImageFromUrl(original, searchResponse["recipeName"], index, recipeResponse["images"])
        if fileName:
            jsonBuilder["Images"] = recipeResponse["images"]
            jsonBuilder["RecipeName"] = searchResponse["recipeName"]
            jsonBuilder["RecipeURL"] = recipeResponse["source"]["sourceRecipeUrl"]
            jsonBuilder["CleanIngredients"] = searchResponse["ingredients"]
            jsonBuilder["RawIngredients"] = recipeResponse["ingredientLines"]
            jsonBuilder["ImagePath"] = fileName
        else:
            return None
    else:
        noImages.append(original+" "+searchResponse["recipeName"])
        return None
    return jsonBuilder           

In [11]:
def searchFromList():
    responses = []
    notFound = []
    foodList = pd.read_csv("food101.csv", sep=',')
    for query in foodList['FoodName']:
        print "query: ", query
        params = query.lower().replace(" ", "+")
        url = baseURL.replace("@",params)
        response = json.loads(requests.get(url).text)
        if not response["matches"]:
            "Hello no matches :3"
            notFound.append(query)
    
        else:
            candidate = bestMatch(query, response)
            candidateID = candidate["id"]
            recipeResponse = getRecipe(candidateID)
            builtJson = buildJson(query, candidate, recipeResponse)
            responses.append(builtJson)
            
    data = json.dumps(responses)
    with open('recipesData.json', 'w') as f:
        json.dump(responses, f)
    with open('notFound.json', 'w') as f:
        json.dump(notFound, f)
    
    return responses, notFound

In [12]:
def searchRecipesRange():
    notFound = []
    foodList = pd.read_csv("food101.csv", sep=',')
    jsonBuilder = {}
    for query in foodList['FoodName']:
        print "query: ", query
        params = query.lower().replace(" ", "+")
        url = baseURL.replace("@",params)
        succeed = False
        #Sometimes, calling the api in such a sort time causes it to shut down
        #Let's not get out of here until we can get the api call
        while not succeed:
            try:
                response = json.loads(requests.get(url).text)
                succeed = True
            except:
                print "Trying to reconnect..."
                jsonBackup = jsonBuilder
                time.sleep(2)
            
        if not response["matches"]:
            "Hello no matches :3"
            notFound.append(query)
    
        else:
            matches = rangeMatch(query, response)
            lst = []
            i = 0
            for match in matches:
                candidateID = match["id"]
                recipeResponse = getRecipe(candidateID)
                builtJson = buildWideJson(query, match, recipeResponse, i)
                if builtJson:
                    lst.append(builtJson)
                    i+=1
                
            print "For the class "+query+" it should be "+str(i)+" images from the "+str(len(matches))+" initially found"
            jsonBuilder[query] = lst
            
    jsonBackup = jsonBuilder
    with open('recipesWideData.json', 'w') as f:
        json.dump(jsonBuilder, f)
    with open('notFoundWide.json', 'w') as f:
        json.dump(notFound, f)
        
    df = pd.DataFrame.from_records(csvList, columns=["path_to_image", "recipeName"])
    df.to_csv("paths.csv", index=False, encoding='utf-8')
    data = json.dumps(jsonBuilder)
    
    return jsonBuilder, notFound

#searchRecipesRange()

In [57]:
def checkCorrespondence():
    with open('recipesWideData.json') as data_file:    
        data = json.load(data_file)
    foodList = pd.read_csv("food101.csv", sep=',')
    for query in foodList['FoodName']:
        lengthJson = len(data[query])
        folderName = (query.replace(" ", "_")).lower()
        DIR="food101Pictures/"+folderName
        lengthImages = len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))])
        status = (lengthJson == lengthImages)
        print"For the query "+query+" the json has "+str(lengthJson)+" recipes and the folder has "+str(lengthImages)+" images, status is: "+str(status)


def checkForErrors():
    with open('recipesWideDataCorrected.json') as data_file:    
        data = json.load(data_file)
    foodList = pd.read_csv("food101.csv", sep=',')
    for query in foodList['FoodName']:
        lengthJson = len(data[query])
        folderName = (query.replace(" ", "_")).lower()
        DIR="food101Pictures/"+folderName
        lst = [name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]
        lst.sort(key=lambda x: os.path.getmtime(DIR+"/"+x))
        lengthImages = len(lst)
        status = (lengthJson == lengthImages)
        i = 0
        for elem in data[query]:
            fileName =  DIR+"/"+lst[i] 
            status = status and (elem["ImagePath"] == fileName)
            i+=1
            
        
        print"For query "+query+" the json has "+str(lengthJson)+" recipes and the folder has "+str(lengthImages)+" images, status is: "+str(status)


In [28]:
def cleanData():
    with open('recipesWideData.json') as data_file:    
        data = json.load(data_file)

    data["Apple pie"].pop(40)
    data["Baby back ribs"].pop(42)
    data["Baby back ribs"].pop(39)
    data["Beef Tacos"].pop(5)
    data["Beef tartare"].pop(39)
    data["Beef tartare"].pop(17)
    data["Beef tartare"].pop(14)
    data["Beef tartare"].pop(1)
    data["Beignets"].pop(16)
    data["Bread pudding"].pop(21)
    data["Breakfast burrito"].pop(41)
    data["Cannoli"].pop(33)
    data["Chicken wings"].pop(36)
    data["Chicken wings"].pop(0)
    data["Churros"].pop(20)
    data["Deviled eggs"].pop(14)
    data["Donuts"].pop(30)
    data["Dumplings"].pop(6)
    data["Eggs benedict"].pop(16)
    data["Escargots"].pop(26)
    data["Filet mignon"].pop(33)
    data["French fries"].pop(35)
    data["Fried calamari"].pop(10)
    data["Fried rice"].pop(36)
    data["Grilled salmon"].pop(45)
    data["Grilled salmon"].pop(5)
    data["Lobster bisque"].pop(26)
    data["Mussels"].pop(30)
    data["Omelette"].pop(20)
    data["Omelette"].pop(13)
    data["Onion rings"].pop(19)
    data["Pad thai"].pop(9)
    data["Oysters"].pop(45)
    data["Paella"].pop(44)
    data["Panna cotta"].pop(41)
    data["Peking duck"].pop(14)
    data["Prime rib"].pop(15)
    data["Ramen"].pop(37)
    data["Ramen"].pop(10)
    data["Sashimi"].pop(19)
    data["Seaweed salad"].pop(48)
    data["Spaghetti bolognese"].pop(49)
    data["Tiramisu"].pop(16)
    data["Tuna tartare"].pop(38)
    data["Tuna tartare"].pop(33)
    data["Tuna tartare"].pop(28)
    data["Tuna tartare"].pop(14)

    with open('recipesWideDataCorrected.json', 'w') as f:
        json.dump(data, f)
            
cleanData()